# Les optimisation en ordonnancement.

# Introduction.

Dans ce notebook, je vais principalement traiter des problèmes d'ordonnancement en usine et en atelier, cependant, on peut transposer les modèles mathématiques à d'autres cas d'utilisation.

Les principaux modèles sont ( En Anglais ) :

- Machine unique ( Single machine sheduling/bottleneck )
- Machines parallèles 
- Ateliers à cheminement unique (Flow Shop)
- Ateliers à cheminements multiples (Job Shop)

Je vais résoudre avec divers solveurs Python.

La planification optimale des tâches est une classe de problèmes d'optimisation liés à la planification. Les entrées de ces problèmes sont une liste de tâches (également appelées processus ou tâches) et une liste de machines (également appelées processeurs ou travailleurs). La sortie requise est un calendrier - une affectation de tâches aux machines. Le calendrier doit optimiser une certaine fonction objectif.


Etude globale proposée par Estelle Derrien - Github15000

*** CREATION EN COURS ***


# Sommaire

- 1. Machine unique
        - Notre problème de base
        - Différence entre minimiser le temps d'exécution, et la somme des poids/priorités d'exécution. 
        - Différence entre minimiser le temps de complétion et le retard global.
        - Modélisation mathématique
        - Résolution avec les solveurs Python
- 2. Machines parrallèles
        - Notre problème de base
        - Modélisation mathématique
        - Résolution
- 3. Ateliers à cheminement unique (Flow Shop)
        - Notre problème de base
        - Modélisation mathématique
        - Résolution avec Python Pulp
- 4. Ateliers à cheminements multiples (Job Shop)
        - Notre problème de base
        - Modélisation mathématique
        - Exemple


# 1. Ordonnancement de Machine unique 

# Notre problème de base.

J'ai une machine, 6 travaux à réaliser dessus. Les travaux ont une durée différente et doivent être délivrés à différentes dates.
Comment minimiser le temps total d'exécution ?

<div style="text-align:center">
<img src="img/single-machine-scheduling.png">
</div>

La machine:

    - Ne peut pas effectuer plus d'une tâche à la fois. (Pas de multi-tâches)
    - A un temps de configuration avant de commencer à effectuer une tâche.
    - Traite un travail après l'autre immédiatement.

Les tâches:

    - Avoir un temps de traitement spécifique.
    - Facultatif : Avoir une priorité spécifique (poids).
    - Sont équivalentes à un seul emploi (c'est-à-dire que toutes les tâches consistent en un seul emploi).




# Modélisation mathématique

Réalisation en cours !

- Soit T  un ensemble de tâches (T1,T2,T3...Tn)
- Soit Dt la durée d'une tâche
- Soit Et la date d'exécution au plus tard d'une tâche (Due date)
- Soit Ct la date de commencement d'une tâche

Programme linéaire :

- Min Σ T * Dt
- Sous les contraintes
- Σ Ct >= Et


# Résolution avec Pulp

In [20]:
import pulp 
import itertools as it

# Tâches
taches = ["tache1","tache2","tache3"]
durées = {2,5,6}
due = {15,9,11}
total_journee = 24

# Implémentation naîve mettant en retard x3 qui est du à 11 heures du matin:

# heures   1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
# tache1   - -
# tache2       - - - - -
# tache3                 - - - - - -

# Implémentation OK , La tâche 1 est dispo avant 15 heures, 
# la tâche 2 est dispo avant 9 heures, La tâche 3 est dispo avant 11 h

# heures   1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
# tache1                         -  -
# tache2   - - - - - 
# tache3             - - - - - -

# Importer la librairie Pulp sous le pseudo p
import pulp as p 

# Créer un programme linéaire de maximisation
Mon_Probleme = p.LpProblem('minimisation_temps_machine_unique', p.LpMinimize)  

# Créer les variables du problème, 
tache1 = p.LpVariable("tache1", 0, None, p.LpBinary)   
tache2 = p.LpVariable("tache2", 0, None, p.LpBinary)   
tache3 = p.LpVariable("tache3", 0, None, p.LpBinary)  

# Ecrire la fonction objectif 
Mon_Probleme +=  2 * tache1 + 5 * tache2 + 6 * tache3

# Ecrire les contraintes

# Résoudre
Mon_Probleme.solve()
# On imprime les variables qui ont leur valeur optimisées
for v in Mon_Probleme.variables():
    print(v.name, "=", v.varValue)
# La valeur de la fonction objective optimisée est imprimée à l'écran
print("Temps minimisé = ", pulp.value(Mon_Probleme.objective))

tache1 = 0.0
tache2 = 0.0
tache3 = 0.0
Temps minimisé =  0.0


In [9]:
import pulp as pl
import pytups as pt

# Get input data
duration = {1: 8, 2: 12, 3: 6, 4: 13, 5: 2, 6: 12, 7: 15, 8: 20, 9: 16, 10: 16, 11: 10, 12: 14, 13: 17, 14: 3, 15: 20, 16: 19, 17: 13, 18: 6, 19: 18, 20: 11}
priority = {1: 4, 2: 5, 3: 6, 4: 8, 5: 4, 6: 2, 7: 5, 8: 6, 9: 1, 10: 8, 11: 8, 12: 4, 13: 4, 14: 8, 15: 9, 16: 11, 17: 6, 18: 10, 19: 6, 20: 9}
duedate = {1: 52, 2: 80, 3: 133, 4: 150, 5: 53, 6: 133, 7: 113, 8: 107, 9: 75, 10: 133, 11: 77, 12: 126, 13: 117, 14: 72, 15: 111, 16: 111, 17: 117, 18: 69, 19: 115, 20: 68}



# Get intermediate paramters, sets.
C_max = sum(duration.values())
periods = range(C_max)
tasks = duration.keys()

# all legal combinations of task-period assignment
jk_all = pt.TupList((t, p) for t in tasks for p in periods)

# we filter the starts that are too late to be possible:
JK = jk_all.filter_list_f(lambda x: x[1] + duration[x[0]] <= C_max)

# we create a set of tasks that can start at time period k
K_j = JK.to_dict(result_col=1)

# all combinations (t, p, p2) such that I start a task j 
# in time period k and is active in period k2
jkk2 = pt.TupList((j, k, k2) for j, k in JK 
				  for k2 in range(k, k + duration[j]))

# given a period k2, what starts affect make it unavailable:
JK_k2 = jkk2.to_dict(result_col=[0, 1])

# given a start (j, k), what periods k2 are made unavailable:
K2_jk = jkk2.to_dict(result_col=2)

# for each possible start: how late will the task be
t_jk = {(j, k): max(duration[j] + k -1 - duedate[j], 0) * 
				priority[j] for j, k in JK}



# model construction with PuLP
model = pl.LpProblem("Scheduling", pl.LpMinimize)
X = pl.LpVariable.dicts(name='start', indexs=JK,
                            lowBound=0, upBound=1, cat=pl.LpInteger)

# objective function:
model += pl.lpSum(X[j, k] * t_jk[j, k] for j, k in JK)

# one and only one start per task
for j in tasks:
    model += pl.lpSum(X[j, k] for k in K_j[j]) == 1

# only one task is active at any moment:
for k2 in periods:
    model += pl.lpSum(X[j, k] for j, k in JK_k2[k2]) == 1

# solve model:
solver = pl.PULP_CBC_CMD(msg=True)
# solver = pl.CPLEX_CMD(msg=True)
model.solve(solver)



# get tasks starts
starts_out = pt.SuperDict(X).vapply(pl.value).clean().keys_l()

solution = [-1 for p in periods]
for j, k in starts_out:
    for p2 in K2_jk[j, k]:
        solution[p2] = j

print('solution is \n{}'.format(solution))



# verify that all periods are filled:
# counting if there is any with a minus -1
pt.TupList(solution).filter_list_f(lambda v: v == -1)

# verify that each task t appears exactly duration[t] times
task_count = pt.SuperDict().fill_with_default(keys=duration.keys())
for period, task in enumerate(solution):
    task_count[task] += 1
task_count.apply(lambda k, v: duration[k] - v).clean()


AttributeError: 'TupList' object has no attribute 'filter_list_f'

# Résolution avec Pyomo

In [ ]:
from pyomo.environ import *
from IPython.display import display
import pandas as pd

def schedule(JOBS):
    
    # create model
    m = ConcreteModel()
    
    # index set to simplify notation
    J = list(JOBS.keys())

    # decision variables
    m.start      = Var(J, domain=NonNegativeReals)
    m.makespan   = Var(domain=NonNegativeReals)
    m.pastdue    = Var(J, domain=NonNegativeReals)
    m.early      = Var(J, domain=NonNegativeReals)
    
    # additional decision variables for use in the objecive
    m.ispastdue  = Var(J, domain=Binary)
    m.maxpastdue = Var(domain=NonNegativeReals)

    # for modeling disjunctive constraints
    m.y = Var(J, J, domain=Binary)
    BigM = 1000  #  max([JOBS[j]['release'] for j in J]) + sum([JOBS[j]['duration'] for j in J])

    m.OBJ = Objective(
        expr = sum([m.pastdue[j] for j in J]),
        sense = minimize
    )

    m.cons = ConstraintList()
    for j in J:
        m.cons.add(m.start[j] >= JOBS[j]['release'])
        m.cons.add(m.start[j] + JOBS[j]['duration'] + m.early[j] == JOBS[j]['due'] + m.pastdue[j])
        m.cons.add(m.pastdue[j] <= m.maxpastdue)
        m.cons.add(m.start[j] + JOBS[j]['duration'] <= m.makespan)
        m.cons.add(m.pastdue[j] <= BigM*m.ispastdue[j])
        for k in J:
            if j < k:
                m.cons.add(m.start[j] + JOBS[j]['duration'] <= m.start[k] + BigM*(1-m.y[j,k]))
                m.cons.add(m.start[k] + JOBS[k]['duration'] <= m.start[j] + BigM*(m.y[j,k]))
    
    SolverFactory('glpk').solve(m)

    for j in J:
        JOBS[j]['start'] = m.start[j]()
        JOBS[j]['finish'] = m.start[j]() + JOBS[j]['duration']
        JOBS[j]['pastdue'] = m.pastdue[j]()
        JOBS[j]['early'] = m.early[j]()
        JOBS[j]['ispastdue'] = m.ispastdue[j]()
        
    # display table of results
    df = pd.DataFrame(JOBS)
    df['Total'] = df.sum(axis=1)
    df.loc[['due','finish','release','start'],'Total'] = ''
    display(df)
        
    return JOBS

JOBS = schedule(JOBS)

    solver 'glpk'


ApplicationError: No executable found for solver 'glpk'